In [ ]:
# !pip install captum

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from geopy.distance import geodesic
from tqdm.notebook import tqdm, trange
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import plotly.express as px
from sklearn.cluster import KMeans
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
from collections import defaultdict
import math

# Preprocessing:

In [2]:
main_df = pd.read_csv('dataset.csv')

print(
    main_df['Codice'].nunique(), main_df['Descrizione'].nunique(), main_df['Nome'].nunique()
    )

main_df = main_df.dropna(subset=['Descrizione'])  # clean
main_df['Timestamp'] = pd.to_datetime(main_df['Timestamp'])
main_df['Timestamp segnale'] = pd.to_datetime(main_df['Timestamp segnale'])

main_df = main_df.sort_values('Timestamp segnale')

# avoid blocks of signals... (take only the first from a continguous group of signals)
main_df = main_df.loc[main_df['Descrizione'] != \
                              main_df['Descrizione'].shift(1)].copy()


main_df = main_df.drop_duplicates(subset=[
    'Descrizione', 'Timestamp segnale'], keep='first')

unique_value_columns = main_df.nunique() == 1  # Identify columns with a single unique value
main_df = main_df.loc[:, ~unique_value_columns]  # Drop these columns

# we are going to use all the features, potentially.
# So there might be an indicator of "missing" value which is feedable to a neural net
main_df.fillna(-1, inplace=True)

print(
    main_df['Codice'].nunique(), main_df['Descrizione'].nunique(), main_df['Nome'].nunique()
    )

600 402 297
580 402 295


# Going beyond time...
_________


Let's look at the Bayes Rule:

$$P(anomaly | event)  =   \frac{ P( event | anomaly) \cdot P( anomaly) }{ P(event)} $$


For stability, we will use:

$$ log[P(anomaly | event)]  =   log[\frac{ P( event | anomaly) \cdot P( anomaly) }{ P(event)}] $$

So the posteriors can be computed as:

$$ log[P(anomaly | event)]  =   log[ P( event | anomaly)] + log[ P( anomaly) ] - log[ P(event) ] $$

or, conversely, the likelihoods can be computed as:

$$ log[P(event | anomaly)]  =   log[ P( anomaly | event)] - log[ P( anomaly) ] + log[ P(event) ] $$



In general, the predictive maintenance task can be see as the task of approximating posteriors (or likelihoods). 

- What really matters is what we exactly mean by the _conditional probabilities_, be they posteriors or likelihoods. 

- Notice that, for many of the definition we choose, any MLP with non linearities can in principle "memorise" the whole dataset and solve this problem given enough training, so we need to be careful and follow the Ockam's razor principle: we will add "information" to the "conditioning event" little by little, and try to see if we have better predictors w.r.t the only-time setting above.

Let's reason on extreme settings: 
-> simplest conditioning event: a flag indicating the event happened (like in the previous setting)
-> simplest posterior to predict: the next time of occurrence of an anomaly (like in the previous setting)
-> most complex conditioning event encoding: the whole dataset before the current time.
-> most complex posterior to predict: the whole dataset after the current time.

We will fix the simplest posterior: the next time of occurrence of an anomaly (like in the previous setting) and try to iteratively complexify the conditioning event encoding...



### First thing: compute the priors:

In [3]:
"""
To compute prior probabilites on events/anomalies, we use the frequentist definition...
"""
# we'll only consider events with a minimum occurrence count:
analysis_treshold = 300

# Grouping by 'Category'
grouped_prior = main_df.groupby('Descrizione')

# Creating a dictionary of DataFrames
priors = {category: group
          for category, group in grouped_prior
          if len(group) > analysis_treshold}

sample_space_dim = 0

for prior in tqdm(priors):
    sample_space_dim += len(priors[prior])

print('sample space dim', sample_space_dim)

# Creating a dictionary of DataFrames
prior_probs = {
    prior[0]: len(prior[1])/sample_space_dim
    for prior in priors.items()}

# verify we have done things right:
summation = 0
for item in prior_probs.values():
    summation += item
assert 0.9999 <= summation <= 1.00001

  0%|          | 0/40 [00:00<?, ?it/s]

sample space dim 28079


### Second: get the data:

In [4]:
# our inferences are going to be made from this cleaned dataset:
main_df = pd.concat([df for df in priors.values()])
main_df.sort_values('Timestamp', inplace=True)

# we differentiate between events and anomalies
anomalies = {}
total_labels = list(priors.keys())

for label in total_labels:
    if 'guasto' in label or \
            'abbassamento' in label or \
            'ancanza' in label or \
            'non presente' in label or \
            'fuori servizio' in label or \
            'Mancata ' in label or \
            'mergenza ' in label or \
            'Guasto ' in label or \
            'insufficiente ' in label or \
            'Perdita' in label:
        anomalies[label] = priors[label]
        del priors[label]

generic_events = priors

print('number of anomalies to analyse: ', len(anomalies),
      'num of events: ', len(generic_events))

number of anomalies to analyse:  15 num of events:  25


### third: categorical encoding...

In [5]:
codes = main_df['Codice'].unique()
print(f' we have {len(codes)} different codes')
code_encoder = {code_tuple[1]: code_tuple[0] for code_tuple in enumerate(codes)}
code_decoder = {code_tuple[0]: code_tuple[1] for code_tuple in enumerate(codes)}

names = main_df['Nome'].unique()
print(f' we have {len(names)} different names')
name_encoder = {name_tuple[1]: name_tuple[0] for name_tuple in enumerate(names)}
name_decoder = {name_tuple[0]: name_tuple[1] for name_tuple in enumerate(names)}

descs = main_df['Descrizione'].unique()
print(f' we have {len(descs)} different descs')
desc_encoder = {desc_tuple[1]: desc_tuple[0] for desc_tuple in enumerate(descs)}
desc_decoder = {desc_tuple[0]: desc_tuple[1] for desc_tuple in enumerate(descs)}

sists = main_df['Sistema'].unique()
print(f' we have {len(sists)} different systems')
sist_encoder = {sist_tuple[1]: sist_tuple[0] for sist_tuple in enumerate(sists)}
sist_decoder = {sist_tuple[0]: sist_tuple[1] for sist_tuple in enumerate(sists)}

comps = main_df['Componente'].unique()
print(f' we have {len(comps)} different components')
comp_encoder = {comp_tuple[1]: comp_tuple[0] for comp_tuple in enumerate(comps)}
comp_decoder = {comp_tuple[0]: comp_tuple[1] for comp_tuple in enumerate(comps)}

 we have 93 different codes
 we have 57 different names
 we have 40 different descs
 we have 8 different systems
 we have 35 different components


### let's play around with a specific anomaly prediction case:

In [6]:
# computing relative times...
def preprocess_chunk(chunk):

    chunk['rel_time'] = chunk.iloc[-1]['Timestamp'] - chunk['Timestamp']
    chunk['rel_time'] = pd.to_timedelta(chunk['rel_time']).apply(lambda x: \
                                                  x.total_seconds() * 10**6 \
                                                  + x.microseconds)
    chunk['rel_time'] = chunk['rel_time'] / 10**6
    chunk.drop(
        columns=['Timestamp', 'Timestamp chiusura', 'Timestamp segnale', 'Posizione'],
        inplace=True)

    chunk['Codice'] = chunk['Codice'].apply(
        lambda code_str: code_encoder[code_str])
    chunk['Nome'] = chunk['Nome'].apply(
        lambda name_str: name_encoder[name_str])
    chunk['Descrizione'] = chunk['Descrizione'].apply(
        lambda desc_str: desc_encoder[desc_str])
    chunk['Sistema'] = chunk['Sistema'].apply(
        lambda sist_str: sist_encoder[sist_str])
    chunk['Componente'] = chunk['Componente'].apply(
        lambda comp_str: comp_encoder[comp_str])

    return chunk


def get_chunks(current_anomaly):

    # List to hold the chunks
    chunks = []

    # Initialize the starting index
    start_idx = 0

    min_len = 5
    # Iterate through the DataFrame to identify chunk boundaries
    for idx, value in tqdm(enumerate(main_df['Descrizione']), total=len(main_df)):
        if value == current_anomaly:
            if idx - start_idx >= min_len:
                chunk = main_df.iloc[start_idx:idx+1].copy()
                chunk = preprocess_chunk(chunk)
                chunks.append(chunk)
            start_idx = idx + 1

    print(f'Generated {len(chunks)} data samples for anomaly: {current_anomaly}')
    print('length of samples: ')
    pd.Series([len(chunk) for chunk in chunks]).describe()

    print('Tranining a scaler for interarrival times...')
    labels = []
    for chunk in tqdm(chunks):
        labels.extend(list(chunk['rel_time'].unique()))

    scaler = RobustScaler()
    intertimes_rs = scaler.fit_transform(np.array(labels).reshape(-1, 1))
    return chunks, scaler

### Core architecture: attention is all you need... (to deal with multiple time-series records...) eventually, with some recurrency....

In [9]:
class SelfAttention(nn.Module):

    def __init__(self, embed_dim, num_heads, dropout):
        super(SelfAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            dropout=dropout)

    def forward(self, x):
        # x: (batch_size, seq_len, embed_dim)
        # Transpose x to match the MultiheadAttention input shape: (seq_len, batch_size, embed_dim)
        x = x.transpose(0, 1)
        attn_output, _ = self.multihead_attn(x, x, x)  # Self-attention
        # Transpose back to (batch_size, seq_len, embed_dim)
        attn_output = attn_output.transpose(0, 1)
        return attn_output


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.drop = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.drop(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


class PredictorBlock(nn.Module):

    def __init__(self,
                 input_dim,
                 embed_dim,
                 num_heads,
                 mlp_hidden_dim,
                 recurrent=False,
                 dropout=0.1):

        super(PredictorBlock, self).__init__()

        self.layer_norm = nn.LayerNorm(input_dim)  # LayerNorm over the input features
        self.encoding_layer = nn.Linear(input_dim, embed_dim)
        self.self_attention = SelfAttention(embed_dim, num_heads, dropout=dropout)
        self.recurrent = recurrent
        if self.recurrent:
            self.gru = nn.GRU(input_size=embed_dim,
                              hidden_size=embed_dim,
                              num_layers=1,
                              batch_first=True)
        self.mlp = MLP(embed_dim, mlp_hidden_dim, 1, dropout)  # Output dimension is 1 for scalar output

    def forward(self, x):
        # x: (batch_size, seq_len, input_dim)
        x = self.layer_norm(x)  # Apply LayerNorm
        x = self.encoding_layer(x)
        x = self.self_attention(x)  # (batch_size, seq_len, embed_dim)
        if self.recurrent:
            x, _ = self.gru(x)
            x = x[:, -1, :]
        else:
            x = x[:, -1]
        output = self.mlp(x)  # (batch_size, 1)
        return output

In [10]:
class AnomalyPredictor(nn.Module):

    def __init__(
            self,
            input_dim,
            embed_dim=16,
            num_heads=1,
            mlp_hidden_dim=8,
            device='cuda:0',
            include_numeric_feats=False,
            recurrent=False,
            dropout=0.3):

        super(AnomalyPredictor, self).__init__()

        self.code_embedding_layer = nn.Embedding(
            num_embeddings=len(codes),
            embedding_dim=7)
        self.name_embedding_layer = nn.Embedding(
            num_embeddings=len(names),
            embedding_dim=6)
        self.desc_embedding_layer = nn.Embedding(
            num_embeddings=len(descs),
            embedding_dim=6)
        self.sist_embedding_layer = nn.Embedding(
            num_embeddings=len(sists),
            embedding_dim=3)
        self.comp_embedding_layer = nn.Embedding(
            num_embeddings=len(comps),
            embedding_dim=6)

        self.include_numeric_feats = include_numeric_feats

        predictor_input_dim = 28
        if self.include_numeric_feats:
            predictor_input_dim += input_dim

        self.predictor_block = PredictorBlock(
            predictor_input_dim,
            embed_dim,
            num_heads,
            mlp_hidden_dim,
            recurrent=recurrent,
            dropout=dropout)

        self.device = device
        self.to(self.device)

    def package_input_chunk(self, chunk):
        code_tensor = self.code_embedding_layer(torch.tensor(chunk['Codice'].values, device=self.device).to(torch.long))
        name_tensor = self.name_embedding_layer(torch.tensor(chunk['Nome'].values, device=self.device).to(torch.long))
        desc_tensor = self.desc_embedding_layer(torch.tensor(chunk['Descrizione'].values, device=self.device).to(torch.long))
        sist_tensor = self.sist_embedding_layer(torch.tensor(chunk['Sistema'].values, device=self.device).to(torch.long))
        comp_tensor = self.comp_embedding_layer(torch.tensor(chunk['Componente'].values, device=self.device).to(torch.long))

        cats_tensor = torch.hstack([
            code_tensor,
            name_tensor,
            desc_tensor,
            sist_tensor,
            comp_tensor
            ])

        if self.include_numeric_feats:
            nums_tensor = torch.tensor(
                chunk.drop(columns=['Codice', 'Nome', 'Descrizione', 'Sistema', 'Componente']).values,
                device=self.device)

            input_tensor = torch.hstack([cats_tensor, nums_tensor])
        else:
            input_tensor = cats_tensor

        return input_tensor.to(torch.float32).to(self.device)

    def forward(self, chunk):

        x = self.package_input_chunk(chunk).unsqueeze(0)
        # assert not torch.all(torch.isnan(x))
        x = self.predictor_block(x)
        # assert not torch.all(torch.isnan(x))
        return x

In [11]:
def run_experiment(hypers):

    chunks, scaler = get_chunks(hypers['name'])

    torch.manual_seed(hypers['seed'])
    predictor = AnomalyPredictor(
        input_dim=157,
        dropout=hypers['dropout'],
        recurrent=hypers['recurrency'],
        include_numeric_feats=hypers['include_numeric_feats'])

    criterion = nn.MSELoss()
    predictor_optimizer = torch.optim.Adam(
        params=predictor.parameters(),
        lr=hypers['learning_rate'])

    run_tags = []

    if hypers['include_numeric_feats']:
        run_tags.append('NUMS')
    if hypers['recurrency']:
        run_tags.append('RECURRENT')
    if hypers['watch']:
        run_tags.append('WATCH')

    wandb.init(
        project="RailRED",
        name=hypers['name']+'num'+str(hypers['include_numeric_feats']),
        config=hypers,
        mode=('online' if hypers['logging'] else 'disabled'),
        tags=run_tags
    )

    if hypers['watch']:
        wandb.watch(
            models=predictor,
            criterion=F.mse_loss,
            log='all',
            log_freq=100,
            log_graph=True)

    errors = []
    for epoch in range(hypers['epochs']):
        for chunk in tqdm(chunks):

            inner_errors = []
            for label in chunk['rel_time'].unique():
                feat = chunk.loc[chunk['rel_time']==label].drop(
                    columns='rel_time').copy()
                label = scaler.transform(np.array(label).reshape(-1, 1))
                if (label <= 1):
                    pred = predictor(feat)
                    label = torch.tensor(label, device='cuda:0').to(torch.float32)
                    err = criterion(input=pred, target=label)

                    predictor_optimizer.zero_grad()
                    err.backward()
                    predictor_optimizer.step()

                    wandb.log({'error': err})
                    inner_errors.append(err.item())

            mean_err = torch.tensor(inner_errors).mean().item()
            wandb.log({'mean error': err})
            errors.append(mean_err)

    wandb.finish()

## First model: 
__________
just give me the set of signals... and use just attention.

In [ ]:
for curr_anom in anomalies.keys():

    logging = True
    include_numeric_feats = False
    recurrency = False

    hypers = {
        'name': curr_anom,
        'logging': logging,
        'learning_rate': 3e-4,
        'dropout': 0.4,
        'seed': 1,
        'epochs': 5,
        'include_numeric_feats': include_numeric_feats,
        'recurrency': recurrency,
        'watch': False
        }

    try:
        run_experiment(hypers)
    except Exception as e:
        print('aio: ', e)

## Second model:
___

let's add recurrency to see if we do better...

In [ ]:
for curr_anom in anomalies.keys():

    logging = True
    include_numeric_feats = False
    recurrency = True

    hypers = {
        'name': curr_anom,
        'logging': logging,
        'learning_rate': 3e-4,
        'dropout': 0.4,
        'seed': 1,
        'epochs': 5,
        'include_numeric_feats': include_numeric_feats,
        'recurrency': recurrency,
        'watch': False
        }

    try:
        run_experiment(hypers)
    except Exception as e:
        print('aio: ', e)

## Third model:
___

let's add other features to see if we do better...

In [ ]:
for curr_anom in anomalies.keys():

    logging = True
    include_numeric_feats = True
    recurrency = False

    hypers = {
        'name': curr_anom,
        'logging': logging,
        'learning_rate': 3e-4,
        'dropout': 0.2,
        'seed': 1,
        'epochs': 5,
        'include_numeric_feats': include_numeric_feats,
        'recurrency': recurrency,
        'watch': True
        }

    try:
        run_experiment(hypers)
    except Exception as e:
        print('aio: ', e)

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


## Fourth model:
___________
Adding recurrency....

In [12]:
for curr_anom in list(anomalies.keys())[: 3]:

    logging = True
    include_numeric_feats = True
    recurrency = True

    hypers = {
        'name': curr_anom,
        'logging': logging,
        'learning_rate': 3e-4,
        'dropout': 0.4,
        'seed': 1,
        'epochs': 5,
        'include_numeric_feats': include_numeric_feats,
        'recurrency': recurrency,
        'watch': True
        }

    try:
        run_experiment(hypers)
    except Exception as e:
        print('aio: ', e)

  0%|          | 0/28079 [00:00<?, ?it/s]

Generated 106 data samples for anomaly: Alta tensione non presente in M1-T4
length of samples: 
Tranining a scaler for interarrival times...


  0%|          | 0/106 [00:00<?, ?it/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jfcevallos. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


...

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

error,▂▂█▁▄▁▂▁▁▁▁▁▂▃▃▂▂▃▃▂▂▄▅▁▇▂▁▁▁▅▁▃▂▂▁▂▁▂▁▂
mean error,▁▆▃▄█▇▇█▄▁▆▃▅▄▆▅▄▅▃▂▅▃▅▅▅▅▅▄▄▃▅▅▅▅▅▅▄▄▄▅
error,0.33045
mean error,0.33045


DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/sacremoses-0.0.43-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
  Obtaining dependency information for captum from https://files.pythonhosted.org/packages/e1/76/b21bfd2c35cab2e9a4b68b1977f7488c246c8cffa31e3361ee7610e8b5af/captum-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00a 0:00:01


In [ ]:
attributions, approximation_error = ig.attribute((input1, input2),
                                                 baselines=(baseline1, baseline2),
                                                 method='gausslegendre',
                                                 return_convergence_delta=True)
output